In [ ]:
import dill
import pandas as pd
import datetime
import time
import pickle
import matplotlib.pyplot as plt

from geopy.geocoders import Nominatim
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector

In [ ]:
df = pd.read_csv('data/ga_sessions.csv', low_memory=False)
df

In [ ]:
df2 = pd.read_csv('data/ga_hits.csv', low_memory=False)
df2

In [ ]:
values_to_select = [
    'sub_car_claim_click', 
    'sub_car_claim_submit_click',
    'sub_open_dialog_click', 
    'sub_custom_question_submit_click',
    'sub_call_number_click', 
    'sub_callback_submit_click', 
    'sub_submit_success',
    'sub_car_request_submit_click'
]

selected_rows = df2[df2['event_action'].isin(values_to_select)][['session_id', 'event_action']]
selected_rows['event_action'] = 1
selected_rows

In [ ]:
selected_rows[selected_rows.duplicated()]

In [ ]:
selected_rows = selected_rows.drop_duplicates()
df = df.merge(selected_rows, on='session_id', how='left')
df

In [ ]:
df['event_action'] = df['event_action'].apply(lambda x: 0 if pd.isnull(x) else x)
df['event_action'] = df['event_action'].astype(int)
df

## Data Preparation

### Data Cleaning

#### Duplicates

In [ ]:
df[df.duplicated()]

#### Missing values

In [ ]:
df.describe(include='all')

In [ ]:
for i in df.columns:
    print('Характеристика', i)
    print('Количество уникальных значений', df[i].nunique())
    print('Список значений: ')
    print(df[i].value_counts())

In [ ]:
df.info(verbose=True, show_counts=True)

In [ ]:
df.isna().sum().sort_values()

In [ ]:
missing_values = ((df.isna().sum() / len(df))*100).sort_values()
print("Процент пропущенных значений: ")
missing_values

In [ ]:
def print_useful_rows_indo(df):
    print('Количество полностью заполненных объектов из всей выборки:', len(df.dropna()))
    print('Процент полностью заполненных объектов из всей выборки:', round(len(df.dropna()) / len(df) * 100, 2))

In [ ]:
df_clean = df.copy()

In [ ]:
print_useful_rows_indo(df_clean)

In [ ]:
df_clean.device_model.value_counts(dropna=False)

In [ ]:
df_clean.device_os.value_counts(dropna=False)

In [ ]:
df_clean.utm_keyword.value_counts(dropna=False)

In [ ]:
df_clean = df_clean.drop(columns = ['device_model', 'utm_keyword'])
df_clean

In [ ]:
df_clean.isna().sum().sort_values()

In [ ]:
for col in df_clean.columns:
    if df_clean[col].dtype == 'object':
        df_clean[col] = df_clean[col].str.lower()

##### device_brand

In [ ]:
df_clean.device_brand.value_counts(dropna=False)

In [ ]:
df_clean.device_browser.value_counts(dropna=False)

In [ ]:
df_clean[df_clean.device_brand.isna()].device_browser.value_counts(dropna=False)

In [ ]:
for i in df_clean[df_clean.device_brand.isna()].device_browser.unique():
    print('Browser -', i)
    print(df_clean[df_clean['device_browser'] == i].device_brand.value_counts(dropna=False))

In [ ]:
df[(df['device_brand'].isna()) & (df['device_browser'] == 'Safari')].device_screen_resolution.value_counts()

In [ ]:
sorted(df[(df['device_brand'].isna()) & (df['device_browser'] == 'Safari')].device_screen_resolution.unique().tolist())

In [ ]:
def set_device_brand(x):
    if pd.notnull(x['device_brand']):
        return x['device_brand']
    elif x['device_browser'] == 'samsung internet':
        return 'samsung'
    elif x['device_browser'] == 'safari':
        return 'apple'
    elif (x['device_category'] == 'desktop') or (int(x['device_screen_resolution'].split('x')[0]) > int(x['device_screen_resolution'].split('x')[1])):
        return 'pc'
    else:
        return '(not set)'

df_clean['device_brand'] = df_clean.apply(set_device_brand, axis=1)

df_clean[df_clean.device_brand.isna()]

In [ ]:
df_clean[df_clean.device_brand.isna()].device_browser.value_counts(dropna=False)

##### device_os

In [ ]:
df_clean.device_os.value_counts(dropna=False)

In [ ]:
brand = dict(df_clean.device_brand.value_counts())
brand

In [ ]:
def clean_device_os(data):
    brand = data['device_brand']
    if pd.isna(data['device_os']):
        if isinstance(brand, str):
            if 'apple' in brand:
                return 'ios'
            elif 'pc' in brand:
                return 'windows'
        return 'android'
    return data['device_os']

df_clean['device_os'] = df_clean.apply(clean_device_os, axis=1)

df_clean.device_os.value_counts(dropna=False)

In [ ]:
df_clean[df_clean['device_category']=='desktop'].device_os.value_counts()

##### utm_adcontent & utm_campaign & utm_source

In [ ]:
df_clean.isna().sum().sort_values()

In [ ]:
df_clean.utm_campaign.describe()

In [ ]:
df_clean.utm_campaign.value_counts(dropna=False)

In [ ]:
df_clean.utm_adcontent.describe()

In [ ]:
df_clean.utm_adcontent.value_counts(dropna=False)

In [ ]:
df_clean.utm_source.describe()

In [ ]:
df_clean.utm_source.value_counts(dropna=False)

In [ ]:
df_clean[(df_clean.utm_adcontent.isna()) & (df_clean.utm_campaign.isna())]

In [ ]:
df_clean[(df_clean.utm_adcontent.isna()) & (df_clean.utm_campaign.isna())].event_action.value_counts()

In [ ]:
df_clean[(df_clean.utm_adcontent.isna()) & (df_clean.utm_campaign.isna()) & (df_clean['event_action']==1)]

In [ ]:
for i in df_clean[df_clean.utm_adcontent.isna()].utm_campaign.unique():
    print('utm_campaign -', i)
    print(df_clean[df_clean['utm_campaign'] == i].utm_adcontent.value_counts(dropna=False).head(3))

In [ ]:
for i in df_clean.utm_source.unique():
    print('utm_source -', i)
    print(df_clean[df_clean['utm_source'] == i].utm_adcontent.value_counts(dropna=False).head(3))

In [ ]:
df_clean[(df_clean['utm_source']=='kjsLglQLzykiRbcDiGcD') & (df_clean.utm_adcontent.isna())]

In [ ]:
df_clean[df_clean.utm_source.isna()]

In [ ]:
df_clean[df_clean.utm_source.isna()].utm_campaign.value_counts(dropna=False)

In [ ]:
df_clean[df_clean.utm_source.isna()].utm_adcontent.value_counts(dropna=False)

In [ ]:
df_clean.utm_medium.value_counts()

In [ ]:
df_clean = df_clean.fillna('other')

In [ ]:
df_clean.utm_adcontent.value_counts(dropna=False)

In [ ]:
print_useful_rows_indo(df_clean)

#### Outliers

In [ ]:
def calculate_outliers(data):
    q25 = data.quantile(0.25)
    q75 = data.quantile(0.75)
    iqr = q75 - q25
    boundaries = (q25 - 1.5 * iqr, q75 + 1.5 * iqr)
    
    return boundaries

##### visit_number

In [ ]:
df_clean.visit_number.describe()

In [ ]:
grouped_data = df_clean.groupby('visit_number')['event_action'].mean()

plt.figure(figsize=(12, 6))
plt.bar(grouped_data.index, grouped_data.values)
plt.xlabel('visit_number')
plt.ylabel('Среднее значение event_action')
plt.title('Зависимость event_action от visit_number')
plt.show()

In [ ]:
df_clean[df_clean['visit_number']<=3].event_action.value_counts(dropna=False)

In [ ]:
df_clean[df_clean['visit_number']>3].event_action.value_counts(dropna=False)

In [ ]:
visit_number_bounds = calculate_outliers(df_clean.visit_number)
visit_number_bounds

### Feature engineering

In [ ]:
df_eng = df_clean.copy()

In [ ]:
df_eng.describe(include='all')

##### date

In [ ]:
df_eng['visit_date'] = pd.to_datetime(df_eng['visit_date'])

df_eng['month'] = df_eng.visit_date.apply(lambda x: x.month)
df_eng['dayofweek'] = df_eng.visit_date.apply(lambda x: x.day)
df_eng['year'] = df_eng.visit_date.apply(lambda x: x.year)

In [ ]:
df_eng.year.value_counts()

In [ ]:
df_eng['visit_time'] = pd.to_datetime(df_eng['visit_time'], format='%H:%M:%S')
df_eng['hour'] = df_eng['visit_time'].dt.hour

In [ ]:
df_eng = df_eng.drop(columns = ['year'])

##### session_id & client_id

In [ ]:
df_session = df_eng[['session_id', 'event_action', 'client_id']].copy()
df_session['x_parts_session'] = df_session['session_id'].str.split('.')
df_session['x_parts_client'] = df_session['client_id'].str.split('.')

df_session['session_x0'] = df_session['x_parts_session'].str[0]
df_session['session_x1'] = df_session['x_parts_session'].str[1]
df_session['session_x2'] = df_session['x_parts_session'].str[2]

df_session['client_x0'] = df_session['x_parts_client'].str[0]
df_session['client_x1'] = df_session['x_parts_client'].str[1]                    

unique_session_x0_count = df_session['session_x0'].nunique()
unique_session_x1_count = df_session['session_x1'].nunique()
unique_session_x2_count = df_session['session_x2'].nunique()

unique_client_x0_count = df_session['client_x0'].nunique()
unique_client_x1_count = df_session['client_x1'].nunique()

print(f"Количество уникальных значений в session_x0: {unique_session_x0_count}")
print(f"Количество уникальных значений в session_x1: {unique_session_x1_count}")
print(f"Количество уникальных значений в session_x2: {unique_session_x2_count}")

print(f"Количество уникальных значений в client_x0: {unique_client_x0_count}")
print(f"Количество уникальных значений в client_x1: {unique_client_x1_count}")


##### device_screen_resolution len

In [ ]:
df_eng['dev_scr_res_len'] = df_eng.apply (lambda x: len(x.device_screen_resolution), axis = 1)
df_eng.dev_scr_res_len.value_counts()

##### browser

In [ ]:
def clean_device_browser(browser):
    if 'instagram' in browser:
        return 'instagram'
    elif 'android' in browser:
        return 'android webview'
    elif 'opera' in browser:
        return 'opera'
    elif 'mozilla' in browser:
        return 'mozilla'
    elif 'internet explorer' in browser:
        return 'edge'
    else:
        return browser

df_eng['device_browser'] = df_eng['device_browser'].apply(clean_device_browser)

##### utm_source & utm_adcontent & utm_campaign & utm_medium & device_browser & device_os & device_brand

In [ ]:
columns_to_filter = ['utm_source', 'utm_medium', 'utm_adcontent', 'utm_campaign',
                         'device_browser', 'device_os', 'device_brand']

for column in columns_to_filter:
    print(df_eng[column].value_counts())

In [ ]:
for column in columns_to_filter:
    counts = df_eng[column].value_counts()
    filt = counts[counts < 750].index
    df_eng[column] = df_eng[column].apply(lambda x: 'rare' if x in filt else x)

In [ ]:
for column in columns_to_filter:
    print(df_eng[column].value_counts())

##### geo_city

In [ ]:
df_eng['is_russia'] = df_eng['geo_country'].apply(lambda x: 1 if x == 'russia' else 0)

In [ ]:
df_eng['is_available'] = df_eng['geo_city'].apply(lambda x: 1 if x == 'moscow' or x == 'saint petersburg' else 0)

In [ ]:
df_eng['geo_country'] = df_eng['geo_country'].apply(lambda x: 'россия' if x == 'russia' else x)

In [ ]:
cities = {
    'tuymazy': 'туймазы',
    'zagorjanskas': 'загорянский',
    'novoye devyatkino': 'новое девяткино',
    'yablonovsky': 'яблоновский',
    'petrovo-dalneye': 'петрово-дальнее',
    'kalininets': 'калининец'
}

df_eng['geo_city'] = df_eng['geo_city'].replace(cities)

In [ ]:
df_eng[(df_eng['geo_city'] == '(not set)') & (df_eng['geo_country'] == '(not set)')].event_action.value_counts()

In [ ]:
df_eng = df_eng[(df_eng['geo_city'] != '(not set)') & (df_eng['geo_country'] != '(not set)')]

In [ ]:
def geocode_cities(df):
    from geopy.geocoders import Nominatim
    
    df_copy = df.copy()

    geolocator = Nominatim(user_agent="geo_app")
    try:
        with open('data/coords_cache.pkl', 'rb') as cache_file:
            coords_cache = pickle.load(cache_file)
    except (FileNotFoundError, EOFError):
        coords_cache = {}

    def get_coordinates(city, coord_type):
        if city not in coords_cache:
            try:
                if city == '(not set)' or city is None:
                    coords_cache[city] = None
                else:
                    location = geolocator.geocode(city)
                    if location:
                        coords_cache[city] = {
                            'latitude': location.latitude,
                            'longitude': location.longitude
                        }
                    else:
                        coords_cache[city] = None
                with open('data/coords_cache.pkl', 'wb') as cache_file:
                    pickle.dump(coords_cache, cache_file)
            except Exception as e:
                print(f"Error geocoding {city}: {str(e)}")
                coords_cache[city] = None

        if coords_cache[city]:
            return coords_cache[city][coord_type]
        else:
            return None

    df_copy['latitude'] = df_copy['geo_city'].apply(lambda x: get_coordinates(x, 'latitude'))
    df_copy['longitude'] = df_copy['geo_city'].apply(lambda x: get_coordinates(x, 'longitude'))

    df_copy['latitude'].fillna(df_copy['geo_country'].apply(
        lambda x: get_coordinates(x, 'latitude')), inplace=True)
    df_copy['longitude'].fillna(df_copy['geo_country'].apply(
        lambda x: get_coordinates(x, 'longitude')), inplace=True)

    return df_copy


df_eng = geocode_cities(df_eng)


In [ ]:
grouped = df_eng.groupby('geo_country')
filtered = grouped.filter(lambda x: (x['event_action'] == 0).all())
countries = filtered['geo_country'].unique()
countries

In [ ]:
df_eng.isna().sum()

#### StandardScaler

In [ ]:
quantitative_variables = ['visit_number', 'month', 'dayofweek', 'hour', 'dev_scr_res_len', 
                          'latitude', 'longitude']

data = df_eng[quantitative_variables].copy()
data

In [ ]:
std_scaler = StandardScaler()
std_scaler.fit(data)
std_scaled = std_scaler.transform(data)
print(std_scaled)
print(std_scaled.shape)

In [ ]:
df_eng[std_scaler.get_feature_names_out()+'_std'] = std_scaled

df_eng.head()

#### OneHotEncoder

In [ ]:
categories = ['utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent',
       'device_category', 'device_brand', 'device_browser', 'device_os']

data = df_eng[categories].copy()
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(data)
ohe_data = ohe.transform(data)
df_eng[ohe.get_feature_names_out()] = ohe_data

In [ ]:
df_eng.shape

#### drop

In [ ]:
df_drop = df_eng.copy()

In [ ]:
columns_for_drop = ['session_id', 'client_id', 'visit_date', 'visit_time', 'visit_number',
       'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent',
       'device_category', 'device_brand', 'device_screen_resolution',
       'device_browser', 'geo_country', 'geo_city', 'month',
       'dayofweek', 'hour', 'dev_scr_res_len', 'latitude',
       'longitude', 'device_os']

df_drop = df_drop.drop(columns_for_drop, axis=1)

In [ ]:
df_drop

In [ ]:
df_drop.to_csv('data/df_drop.csv', index=False)

## Modeling